In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# import json

In [ ]:
# # 匯入tweets_DM.json
# # /content/drive/MyDrive/datamining/tweets_DM.json

# file = '/content/drive/MyDrive/datamining/tweets_DM.json'
# tweets = []
# for line in open(file, 'r'):
#     tweets.append(json.loads(line))

In [ ]:
# tweets[:5]

In [ ]:
# len(tweets)

In [ ]:
# from pandas import json_normalize
# df = json_normalize(tweets)

In [ ]:
# df.columns = ['score','index', 'crawldate', 'type', 'hashtags', 'id', 'text']
# print("Shape of df: ", df.shape)

In [ ]:
# pip install emoji

In [ ]:
# import emoji
# def deemoji(a):
#     return emoji.demojize(a, delimiters=("", ""))

In [ ]:
# df['text'] = df['text'].map(deemoji)

In [ ]:
# df.columns

In [ ]:
# new_df

---
# Implementation

In [1]:
import pandas as pd

test_data = pd.read_csv('colab_test.csv')
train_data = pd.read_csv('colab_train.csv')

In [2]:
from transformers import RobertaTokenizer

In [3]:
MODEL_NAME = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

In [4]:
def roberta_encode(texts, tokenizer):
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(text)
        
        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])
        
        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN
        
        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')
        
        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1
    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [5]:
X_data = train_data['text_2'].to_numpy().reshape(-1)
y_data = train_data['emotion'].to_numpy().reshape(-1)

In [6]:
# Transform categories into numbers
categories = train_data['emotion'].values.reshape(-1)

category_to_id_ = {}
category_to_name_ = {}

for index, c in enumerate(y_data):
    if c in category_to_id_:
        category_id_ = category_to_id_[c]
    else:
        category_id_ = len(category_to_id_)
        category_to_id_[c] = category_id_
        category_to_name_[category_id_] = c
    
    y_data[index] = category_id_

# Display dictionary
category_to_name_

{0: 'anticipation',
 1: 'sadness',
 2: 'fear',
 3: 'joy',
 4: 'anger',
 5: 'trust',
 6: 'disgust',
 7: 'surprise'}

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=777)

In [8]:
import numpy as np
MAX_LEN = 256

X_train = roberta_encode(X_train, tokenizer)
X_test = roberta_encode(X_test, tokenizer)

In [9]:
y_train = np.asarray(y_train, dtype='int32')
y_test = np.asarray(y_test, dtype='int32')

In [10]:
def build_model(n_categories):
    input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
    input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
    input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

    # Import RoBERTa model from HuggingFace
    roberta_model = TFRobertaModel.from_pretrained(MODEL_NAME)
    x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

    # Huggingface transformers have multiple outputs, embeddings are the first one,
    # so let's slice out the first position
    x = x[0]

    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(n_categories, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    return model

In [11]:
from transformers import TFRobertaModel
import tensorflow as tf

In [12]:
categories = train_data['emotion'].unique()
n_categories = len(categories)
model = build_model(n_categories)
model.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 256)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_word_ids[0][0]',         
 el)                            thPoolingAndCrossAt               'input_mask[0][0]',         

C:\Users\user\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [12]:
MODEL_NAME = 'roberta-base'

BATCH_SIZE = 1
EPOCHS = 1

print('Training...')
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test))


Training...
  20359/1018894 [..............................] - ETA: 711:45:03 - loss: 1.4489 - accuracy: 0.4685

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\user\AppData\Local\Temp/ipykernel_13148/1658558538.py", line 7, in <module>
    history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test))
  File "C:\Users\user\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\user\anaconda3\lib\site-packages\keras\engine\training.py", line 1216, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 910, in __call__
    result = self._call(*args, **kwds)
  File "C:\

TypeError: object of type 'NoneType' has no len()

---
# Thoughts

    Try many times...
    but still cannot finish it due to without GPU :(
    (I have tried many time to install GPU, but the version still incompatitble.)

![title](cannot_fininsh.png)